In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def family_survival_prediction(train_data, test_data):
    # Identify passengers with the same ticket and family name
    def get_family_group(row):
        return f"{row['Ticket']}_{row['Name'].split(',')[0]}"
    
    train_data['FamilyGroup'] = train_data.apply(get_family_group, axis=1)
    test_data['FamilyGroup'] = test_data.apply(get_family_group, axis=1)
    
    # Predictions based on the two rules
    predictions = np.zeros(len(test_data), dtype=int)
    
    for i, passenger in test_data.iterrows():
        # Rule 1: Predict survive for males titled "Master" with surviving family
        if (passenger['Title'] == 'Master' and 
            passenger['Sex'] == 'male'):
            # Check if entire family (excluding adult males) survives
            family_group_data = train_data[train_data['FamilyGroup'] == passenger['FamilyGroup']]
            adult_males = family_group_data[(family_group_data['Sex'] == 'male') & (family_group_data['Age'] > 16)]
            
            if len(adult_males) == 0:  # No adult males in the group
                other_family = family_group_data[
                    (family_group_data['Sex'] == 'female') | 
                    ((family_group_data['Sex'] == 'male') & (family_group_data['Age'] <= 16))
                ]
                
                if len(other_family) > 0 and (other_family['Survived'] == 1).all():
                    predictions[i] = 1
        
        # Rule 2: Predict die for females with entire family dying
        elif passenger['Sex'] == 'female':
            family_group_data = train_data[train_data['FamilyGroup'] == passenger['FamilyGroup']]
            adult_males = family_group_data[(family_group_data['Sex'] == 'male') & (family_group_data['Age'] > 16)]
            
            if len(adult_males) == 0:  # No adult males in the group
                other_family = family_group_data[
                    (family_group_data['Sex'] == 'female') | 
                    ((family_group_data['Sex'] == 'male') & (family_group_data['Age'] <= 16))
                ]
                
                if len(other_family) > 0 and (other_family['Survived'] == 0).all():
                    predictions[i] = 0
    
    return predictions

# Perform standard preprocessing (as in previous code)
def prepare_data(train_data, test_data):
    # [Previous preprocessing steps remain the same]
    train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
    train_data['Title'] = train_data['Title'].replace(rare_titles, 'Rare')
    test_data['Title'] = test_data['Title'].replace(rare_titles, 'Rare')
    
    train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
    train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
    test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
    test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
    # Age imputation
    title_age_median = train_data.groupby('Title')['Age'].median()
    train_data['Age'] = train_data.apply(
        lambda row: title_age_median[row['Title']] if pd.isna(row['Age']) else row['Age'], 
        axis=1
    )
    test_data['Age'] = test_data.apply(
        lambda row: title_age_median.get(row['Title'], train_data['Age'].median()) 
        if pd.isna(row['Age']) else row['Age'], 
        axis=1
    )
    
    # Other preprocessing steps
    train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
    train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())
    test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())
    
    return train_data, test_data

# Prepare data
train_data, test_data = prepare_data(train_data, test_data)
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)
# Apply family survival rules prediction
family_survival_predictions = family_survival_prediction(train_data, test_data)

# For passengers without rule-based predictions, use Random Forest
remaining_mask = (family_survival_predictions == 0)
X = pd.get_dummies(train_data[["Pclass", "Sex",  "SibSp", "Parch", "Embarked", "Title", "FamilySize","IsAlone","Fare"]])
y = train_data["Survived"]

# Prepare test data
X_test = pd.get_dummies(test_data[["Pclass", "Sex",  "SibSp", "Parch", "Embarked", "Title", "FamilySize","IsAlone","Fare"]])

# Train Random Forest for remaining predictions
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=43)
rf_model.fit(X, y)

# Predict remaining passengers
rf_predictions = rf_model.predict(X_test[remaining_mask])

# Combine predictions
final_predictions = family_survival_predictions.copy()
final_predictions[remaining_mask] = rf_predictions

# Create submission
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': final_predictions})
output.to_csv('submission.csv', index=False)
print("Submission saved successfully!")


Submission saved successfully!


In [5]:
# from sklearn.model_selection import GridSearchCV

# # Définir les paramètres à tester
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 5, 7, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Créer le modèle de base
# rf = RandomForestClassifier(random_state=43)

# # Recherche par grille avec validation croisée
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
#                            cv=5, scoring='accuracy', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # Meilleurs paramètres
# print("Meilleurs paramètres:", grid_search.best_params_)
# print("Meilleure précision:", grid_search.best_score_)
